## How to Use This Notebook

This Jupyter notebook uses the `ipywidgets` module to create a graphical user interface. Here's how to use it:

1. **Run the interface**: The cell below imports the `pygui` module and calls the `create_interface` function. This function creates the graphical user interface. To run this cell, click on it to select it, then click the "Run" button or use the `Shift + Enter` shortcut or the Play button in the top menu.

2. **First steps**: To import data , you need to first replace `user` with your Solforge Fusion `username` in the box below. Then click the `Load` button on the bottom of the notebook. 

In [1]:
import pygui as pgui
pgui.setup_interface()
change1 = {'name': 'value', 'old': '', 'new': 'Spellknight', 'owner': pgui.cardTypes_names_widget['Creature/Spell']}
change2 = {'name': 'value', 'old': 'Chaining', 'new': '', 'owner': pgui.cardTypes_names_widget['Modifier']}
#pgui.update_filter_widget(change1)
#pgui.update_filter_widget(change2)

Getting entity names for Modifier
Entity names for Modifier: ['Abetting', 'Abyssal', 'Adaptive', 'Ardent', 'Armored', 'Arrogant', 'Assailing', 'Biding', 'Blitzing', 'Bolstering', 'Brashful', 'Camouflaged', 'Chaining', 'Charge Plated', 'Combat', 'Commanding', 'Corrupt', 'Cutthroat', 'Devouring', 'Doom', 'Ferocious', 'Fleet', 'Forsaken', 'Ghoulish', 'Hematophagic', 'Hibernating', 'Howling', 'Inflaming', 'Kindred', 'Magnivorous', 'Mystical', 'Necrotic', 'Pack', 'Petrifying', 'Piercing', 'Prepared', 'Progressive', 'Protective', 'Pyromantic', 'Quaking', 'Raging', 'Rallying', 'Reckless', 'Recycling', 'Refreshing', 'Regenerating', 'Reinforced', 'Roaring', 'Rooted', 'Salvage', 'Sated', 'Shambling', 'Shieldbearing', 'Shifting', 'Solemn', 'Soothing', 'Sparking', 'Stabilizing', 'Stampeding', 'Steadfast', 'Stout', 'Summoning', 'Suppressing', 'Synapse', 'Taunting', 'Tormenting', 'Toxic', 'Unstable', 'Vampiric', 'Venerable', 'Vile', 'Volcanic']
Adding Modifier widget
Getting entity names for Creatur

Output()

Output()

Button(description='Show Graph', style=ButtonStyle())

Getting entity names for Modifier
Entity names for Modifier: ['Abetting', 'Abyssal', 'Adaptive', 'Ardent', 'Armored', 'Arrogant', 'Assailing', 'Biding', 'Blitzing', 'Bolstering', 'Brashful', 'Camouflaged', 'Chaining', 'Charge Plated', 'Combat', 'Commanding', 'Corrupt', 'Cutthroat', 'Devouring', 'Doom', 'Ferocious', 'Fleet', 'Forsaken', 'Ghoulish', 'Hematophagic', 'Hibernating', 'Howling', 'Inflaming', 'Kindred', 'Magnivorous', 'Mystical', 'Necrotic', 'Pack', 'Petrifying', 'Piercing', 'Prepared', 'Progressive', 'Protective', 'Pyromantic', 'Quaking', 'Raging', 'Rallying', 'Reckless', 'Recycling', 'Refreshing', 'Regenerating', 'Reinforced', 'Roaring', 'Rooted', 'Salvage', 'Sated', 'Shambling', 'Shieldbearing', 'Shifting', 'Solemn', 'Soothing', 'Sparking', 'Stabilizing', 'Stampeding', 'Steadfast', 'Stout', 'Summoning', 'Suppressing', 'Synapse', 'Taunting', 'Tormenting', 'Toxic', 'Unstable', 'Vampiric', 'Venerable', 'Vile', 'Volcanic']
Filtered options for Spellknight: ['Chaining', 'Venerab

RecursionError: maximum recursion depth exceeded while calling a Python object